In [ ]:
import os
import sys
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import scipy
import glob
import cv2
import liblzfse
import configparser

dep_scale = 0.2
num_skip = 10
refcam = "leftcam"
#seqname_refcam = 'human-dualrig002-properlyscaledcams-leftcam-ds-finetune-fgbkgd'
seqname_refcam = 'catamelie-dualrig002-properlyscaledcams-leftcam-ds-finetune-fgbkgd'

In [ ]:
def load_root(root_dir, cap_frame):
    """
    load all the root se(3)
    input is ...-(00000.txt)
    """
    camlist = []
    #cam_path = '%s0*.txt'%(root_dir)
    cam_path = '%s*.txt'%(root_dir)
    all_path = sorted(glob.glob(cam_path))
    if cap_frame>0:
        all_path = all_path[:cap_frame]
    for idx,path in enumerate(all_path):
        rtk = np.loadtxt(path)
        camlist.append(rtk)
    camlist = np.asarray(camlist)
    return camlist

def read_depth(filepath, height=256, width=192):
    with open(filepath, 'rb') as depth_fh:
        raw_bytes = depth_fh.read()
        decompressed_bytes = liblzfse.decompress(raw_bytes)
        depth_img = np.frombuffer(decompressed_bytes, dtype=np.float32)

    #depth_img = depth_img.reshape((640, 480))      # For a FaceID camera 3D Video
    depth_img = depth_img.copy().reshape((height, width))  # For a LiDAR 3D Video

    return depth_img

def read_conf(filepath, height=256, width=192):
    with open(filepath, 'rb') as conf_fh:
        raw_bytes = conf_fh.read()
        decompressed_bytes = liblzfse.decompress(raw_bytes)
        conf_img = np.frombuffer(decompressed_bytes, dtype=np.uint8)

    #conf_img = conf_img.reshape((640, 480))        # For a FaceID camera 3D Video
    conf_img = conf_img.copy().reshape((height, width))    # For a LiDAR 3D Video

    return conf_img

In [ ]:
def correspondenceGUI(refcam_image, secondcam_image, refcam_conf, secondcam_conf, alpha = 0.7):
    # INPUTS:
    # 1. refcam_image:          shape = (H, W, 3)
    # 2. secondcam_image:       shape = (H, W, 3)
    # 3. refcam_conf:           shape = (H, W)
    # 4. secondcam_conf:        shape = (H, W)
    #
    # RETURNS:
    # 1. xy_coords_refcam:     shape = (N, 2)
    # 2. xy_coords_secondcam:    shape = (N, 2)

    red_image = np.zeros_like(refcam_image)
    red_image[:, :, 0] = 255.
    refcam_image = refcam_image * (1 - (refcam_conf[..., np.newaxis] < 1.5) * alpha) +  ((refcam_conf[..., np.newaxis] < 1.5) * alpha) * red_image
    secondcam_image = secondcam_image * (1 - (secondcam_conf[..., np.newaxis] < 1.5) * alpha) +  ((secondcam_conf[..., np.newaxis] < 1.5) * alpha) * red_image
    refcam_image = refcam_image.astype(int)
    secondcam_image = secondcam_image.astype(int)
    
    leftright_image = np.concatenate([refcam_image, secondcam_image], axis = 1)         # shape = (H, 2W, 3)
    
    f = plt.figure(figsize=(36, 24))
    ax = f.add_subplot(111) 
    ax.imshow(leftright_image)
    ax.set_title('Select at least 6 pairs of corresponding points in the refcam and secondcam images')
    ax.set_axis_off()

    xy_coords_refcam = []
    xy_coords_secondcam = []

    while True:
        #plt.sca(ax1)
        # one pair
        xy_coords = plt.ginput(2, mouse_stop=3, timeout = 0)        # a list of tuples of (x,y) coordinates (mouse_stop = 3 refers to RIGHTCLICK to exit)

        # no coordinates returned: exit while loop
        if len(xy_coords) == 0:
            break
        
        xy_coords = np.array(xy_coords).astype(int)                 # array.shape = (N = 2, 2)

        # check whether the gt depth of the corresponding points of the leftcam / rightcam image has conf. score of 2
        xy_coord_refcam = xy_coords[0, ...].copy()
        xy_coord_secondcam = xy_coords[1, ...].copy()

        xy_coord_secondcam[0] = xy_coord_secondcam[0] - refcam_image.shape[1]
        refcam_conf_at_xy = refcam_conf[xy_coord_refcam[1], xy_coord_refcam[0]]
        secondcam_conf_at_xy = secondcam_conf[xy_coord_secondcam[1], xy_coord_secondcam[0]]

        if refcam_conf_at_xy < 1.5 or secondcam_conf_at_xy < 1.5:
            print("this point doesn't have a high confidence, choose again")
            continue
        else:
            ax.plot(xy_coords[:, 0], xy_coords[:, 1], 'r*', markersize=10, linewidth=2)
            print("choose next pair of corresponding points")
        
        xy_coords_refcam.append(xy_coord_refcam)
        xy_coords_secondcam.append(xy_coord_secondcam)

    plt.close(f)
    return xy_coords_refcam, xy_coords_secondcam

In [ ]:
if refcam == "leftcam":
    secondcam = "rightcam"
if refcam == "rightcam":
    secondcam = "leftcam"

seqname_secondcam = seqname_refcam.replace(refcam, secondcam)

In [ ]:
# extract intrinsics for refcam
rootdir_bkgd_refcam = "logdir/{}/obj1/".format(seqname_refcam)
rtks_refcam = load_root(rootdir_bkgd_refcam, 0)  # cap frame=0=>load all
ks_refcam = rtks_refcam[0, 3, :]
K_refcam = np.array([[ks_refcam[0], 0., ks_refcam[2]],
                     [0., ks_refcam[1], ks_refcam[3]],
                     [0., 0., 1.]])

# extract intrinsics for secondcam
config = configparser.RawConfigParser()
config.read('configs/%s.config'%seqname_secondcam)
ks_secondcam = np.array(config.get('data_0', 'ks').split(" ")).astype(float)
K_secondcam = np.array([[ks_secondcam[0], 0., ks_secondcam[2]],
                        [0., ks_secondcam[1], ks_secondcam[3]],
                        [0., 0., 1.]])

gt_imgpath_secondcam = config.get('data_0', 'datapath1')
gt_imgpath_refcam = gt_imgpath_secondcam.replace(secondcam, refcam)

gt_conf_secondcam = gt_imgpath_secondcam.replace("JPEGImages", "ConfidenceMaps")
gt_conf_refcam = gt_imgpath_refcam.replace("JPEGImages", "ConfidenceMaps")

gt_imglist_secondcam = sorted(glob.glob(os.path.join(gt_imgpath_secondcam, "*.jpg")))
gt_imglist_refcam = sorted(glob.glob(os.path.join(gt_imgpath_refcam, "*.jpg")))

gt_conflist_secondcam = sorted(glob.glob(os.path.join(gt_conf_secondcam, "*.conf")))
gt_conflist_refcam = sorted(glob.glob(os.path.join(gt_conf_refcam, "*.conf")))

In [ ]:
counter = 0
num_corresp = 0
rvecs = []
tvecs = []

P_refcams = []
P_secondcams = []
p_refcams = []
p_secondcams = []

for (gt_imgpath_refcam, gt_imgpath_secondcam) in zip(gt_imglist_refcam, gt_imglist_secondcam):
    print("counter = {}".format(counter))
    if counter % num_skip == 0:

        refcam_image = cv2.imread(gt_imgpath_refcam)[:,:,::-1]
        secondcam_image = cv2.imread(gt_imgpath_secondcam)[:,:,::-1]

        gt_confpath_refcam = gt_imgpath_refcam.replace("JPEGImages", "ConfidenceMaps").replace(".jpg", ".conf")
        gt_confpath_secondcam = gt_imgpath_secondcam.replace("JPEGImages", "ConfidenceMaps").replace(".jpg", ".conf")
        gt_deppath_refcam = gt_imgpath_refcam.replace("JPEGImages", "DepthMaps").replace(".jpg", ".depth")
        gt_deppath_secondcam = gt_imgpath_secondcam.replace("JPEGImages", "DepthMaps").replace(".jpg", ".depth")
        
        refcam_conf = read_conf(gt_confpath_refcam)
        secondcam_conf = read_conf(gt_confpath_secondcam)
        
        refcam_depth = read_depth(gt_deppath_refcam)
        #refcam_depth_rnd = glob.glob("logdir/{}/nvs-inputview-dph_%05d.png".format(seqname_refcam)
        secondcam_depth = read_depth(gt_deppath_secondcam)

        refcam_conf[np.isnan(refcam_depth)] = 0.
        refcam_depth[np.isnan(refcam_depth)] = 4.
        ################## ignoring pixels whose depth values are close to camera ############
        refcam_conf[refcam_depth > 2.0] = 0.
        ######################################################################################
        refcam_depth = refcam_depth * dep_scale

        secondcam_conf[np.isnan(secondcam_depth)] = 0.
        secondcam_depth[np.isnan(secondcam_depth)] = 4.
        ################## ignoring pixels whose depth values are close to camera ############
        secondcam_conf[secondcam_depth > 2.0] = 0.
        ######################################################################################
        secondcam_depth = secondcam_depth * dep_scale

        refcam_conf = cv2.resize(refcam_conf, refcam_image.shape[:2][::-1], interpolation=cv2.INTER_NEAREST)
        secondcam_conf = cv2.resize(secondcam_conf, secondcam_image.shape[:2][::-1], interpolation=cv2.INTER_NEAREST)
        refcam_depth = cv2.resize(refcam_depth, refcam_image.shape[:2][::-1], interpolation=cv2.INTER_LINEAR)
        secondcam_depth = cv2.resize(secondcam_depth, secondcam_image.shape[:2][::-1], interpolation=cv2.INTER_LINEAR)

        # 1. manually annotate correspondences (at least 6)
        xy_coords_refcam, xy_coords_secondcam = correspondenceGUI(refcam_image, secondcam_image, refcam_conf, secondcam_conf)
        
        if len(xy_coords_refcam) == 0 and len(xy_coords_secondcam) == 0:
            counter += 1
            continue
        
        xy_coords_refcam = np.stack(xy_coords_refcam, axis = 0)
        xy_coords_secondcam = np.stack(xy_coords_secondcam, axis = 0)

        # 2. compute relative transformation between leftcam and rightcam
        p_refcam = xy_coords_refcam
        p_secondcam = xy_coords_secondcam
        
        x_coord_refcam = xy_coords_refcam[:, 0]
        y_coord_refcam = xy_coords_refcam[:, 1] 
        p_homo_refcam = np.stack([x_coord_refcam, y_coord_refcam, np.ones_like(x_coord_refcam)], axis=-1)                                       # shape = (N, 3)
        depth_refcam = refcam_depth[y_coord_refcam, x_coord_refcam]                                                                             # shape = (N)

        x_coord_secondcam = xy_coords_secondcam[:, 0]
        y_coord_secondcam = xy_coords_secondcam[:, 1] 
        p_homo_secondcam = np.stack([x_coord_secondcam, y_coord_secondcam, np.ones_like(x_coord_secondcam)], axis=-1)                           # shape = (N, 3)
        depth_secondcam = secondcam_depth[y_coord_secondcam, x_coord_secondcam]                                                                 # shape = (N)
        
        P_refcam = np.repeat(depth_refcam[:, np.newaxis], 3, axis = -1) * np.matmul(p_homo_refcam, np.linalg.inv(K_refcam.T))                   # shape = (N, 3)
        P_secondcam = np.repeat(depth_secondcam[:, np.newaxis], 3, axis = -1) * np.matmul(p_homo_secondcam, np.linalg.inv(K_secondcam.T))       # shape = (N, 3)

        p_refcams.append(p_refcam)
        p_secondcams.append(p_secondcam)

        P_refcams.append(P_refcam)
        P_secondcams.append(P_secondcam)

        num_corresp += P_secondcam.shape[0]

        '''
        _, rvec_init, tvec_init = cv2.solvePnP(P_secondcam[..., np.newaxis].astype('float'), 
                                               p_refcam[..., np.newaxis].astype('float'), 
                                               K_refcam, 
                                               0,
                                               flags=cv2.SOLVEPNP_DLS)

        _, rvec_finetuned, tvec_finetuned = cv2.solvePnP(P_secondcam[..., np.newaxis].astype('float'),
                                                        p_refcam[..., np.newaxis].astype('float'),
                                                        K_refcam,
                                                        0,
                                                        rvec_init,
                                                        tvec_init,
                                                        useExtrinsicGuess=True,
                                                        flags=cv2.SOLVEPNP_ITERATIVE)
        
        # compute reprojection error
        reprojected_p_refcam, _, = cv2.projectPoints(P_secondcam[..., np.newaxis].astype('float'),
                                                     rvec_finetuned,
                                                     tvec_finetuned,
                                                     K_refcam,
                                                     0)
                
        reprojected_p_refcam = reprojected_p_refcam[:, 0, :]                                    # reprojected_p_refcam.shape = (N, 1, 2)
        reproj_err = np.mean(np.linalg.norm(reprojected_p_refcam - p_refcam, axis = -1))
        #reproj_err = np.mean(np.linalg.norm(reprojected_p_refcam - p_normrefcam, axis = -1))
        print("averaged reprojection error [units: pixels]: {}".format(reproj_err))
        '''

        #rvecs.append(rvec_finetuned)
        #tvecs.append(tvec_finetuned)

    #if counter // num_skip == 4:
    #    break

    counter += 1

    if num_corresp > 30:
        break

"""
rvec = np.mean(np.stack(rvecs, axis = 0), axis = 0)
tvec = np.mean(np.stack(tvecs, axis = 0), axis = 0)

secondcam2refcam_rot = cv2.Rodrigues(rvec)[0]
secondcam2refcam = np.eye(4)
secondcam2refcam[:3, :3] = secondcam2refcam_rot                                     # shape = (3, 3)
secondcam2refcam[:3, 3:4] = tvec                                                    # shape = (3, 1)
refcam2secondcam = np.linalg.inv(secondcam2refcam)
"""

In [ ]:
p_refcam = np.concatenate(p_refcams, axis=0)
p_secondcam = np.concatenate(p_secondcams, axis=0)

P_refcam = np.concatenate(P_refcams, axis=0)
P_secondcam = np.concatenate(P_secondcams, axis=0)

In [ ]:
# (version 1) USING ALL POINTS FOR SOLVING refcam2second directly

# compute relative transformation between rightcam and leftcam
_, rvec_init, tvec_init = cv2.solvePnP(P_refcam[..., np.newaxis].astype('float'), 
                                        p_secondcam[..., np.newaxis].astype('float'), 
                                        K_secondcam, 
                                        0,
                                        flags=cv2.SOLVEPNP_DLS)

_, rvec_finetuned, tvec_finetuned = cv2.solvePnP(P_refcam[..., np.newaxis].astype('float'),
                                                p_secondcam[..., np.newaxis].astype('float'),
                                                K_secondcam,
                                                0,
                                                rvec_init,
                                                tvec_init,
                                                useExtrinsicGuess=True,
                                                flags=cv2.SOLVEPNP_ITERATIVE)

# compute reprojection error
reprojected_p_refcam, _, = cv2.projectPoints(P_refcam[..., np.newaxis].astype('float'),
                                                rvec_finetuned,
                                                tvec_finetuned,
                                                K_secondcam,
                                                0)
        
reprojected_p_refcam = reprojected_p_refcam[:, 0, :]        # reprojected_p_refcam.shape = (N, 1, 2)
reproj_err = np.mean(np.linalg.norm(reprojected_p_refcam - p_secondcam, axis = -1))
print("averaged reprojection error [units: pixels]: {}".format(reproj_err))

refcam2secondcam_rot = cv2.Rodrigues(rvec_finetuned)[0]
refcam2secondcam = np.eye(4)
refcam2secondcam[:3, :3] = refcam2secondcam_rot                                             # shape = (3, 3)
refcam2secondcam[:3, 3:4] = tvec_finetuned                                                  # shape = (3, 1)

In [ ]:
# (version 2) USING ALL POINTS FOR SOLVING second2refcam first and then getting refcam2secondcam

# normalize p_refcam and K_refcam
centroid = np.mean(p_refcam, axis = 0, keepdims=True)
scale = np.sqrt(2) / np.mean(np.linalg.norm(p_refcam - centroid, axis = -1))
scaled_centroid = centroid * scale
shift = -scaled_centroid
K_refcam2norm = np.array([[scale, 0,        shift[0, 0]],
                            [0,     scale,    shift[0, 1]],
                            [0,     0,        1]])
x_coord_refcam = p_refcam[:, 0]
y_coord_refcam = p_refcam[:, 1]
p_homo_refcam = np.stack([x_coord_refcam, y_coord_refcam, np.ones_like(x_coord_refcam)], axis=-1)                                       # shape = (N, 3)
p_homo_normrefcam = np.matmul(p_homo_refcam, K_refcam2norm.T)                                                                           # shape = (N, 3)
p_normrefcam = p_homo_normrefcam[:, :2]
K_normrefcam = np.matmul(K_refcam2norm, K_refcam)                                                                                       # shape = (3, 3)

"""
# compute relative transformation between leftcam and rightcam
_, rvec_init, tvec_init = cv2.solvePnP(P_secondcam[..., np.newaxis].astype('float'), 
                                        p_normrefcam[..., np.newaxis].astype('float'), 
                                        K_normrefcam, 
                                        0,
                                        flags=cv2.SOLVEPNP_DLS)

_, rvec_finetuned, tvec_finetuned = cv2.solvePnP(P_secondcam[..., np.newaxis].astype('float'),
                                                p_normrefcam[..., np.newaxis].astype('float'),
                                                K_normrefcam,
                                                0,
                                                rvec_init,
                                                tvec_init,
                                                useExtrinsicGuess=True,
                                                flags=cv2.SOLVEPNP_ITERATIVE)
"""

# cv2.solvePnPRansac -> retval, rvec, tvec, inliers
_, rvec_init, tvec_init, _ = cv2.solvePnPRansac(P_secondcam[..., np.newaxis].astype('float'), 
                                        p_normrefcam[..., np.newaxis].astype('float'), 
                                        K_normrefcam, 
                                        0,
                                        reprojectionError = 0.02,
                                        flags=cv2.SOLVEPNP_DLS)

# cv2.solvePnPRansac -> retval, rvec, tvec, inliers
_, rvec_finetuned, tvec_finetuned, _ = cv2.solvePnPRansac(P_secondcam[..., np.newaxis].astype('float'),
                                                p_normrefcam[..., np.newaxis].astype('float'),
                                                K_normrefcam,
                                                0,
                                                rvec_init,
                                                tvec_init,
                                                useExtrinsicGuess=True,
                                                reprojectionError = 0.02,
                                                flags=cv2.SOLVEPNP_ITERATIVE)

# compute reprojection error
reprojected_p_refcam, _, = cv2.projectPoints(P_secondcam[..., np.newaxis].astype('float'),
                                                rvec_finetuned,
                                                tvec_finetuned,
                                                K_normrefcam,
                                                0)
                                                #K_refcam,
                                                #0)
        
reprojected_p_refcam = reprojected_p_refcam[:, 0, :]        # reprojected_p_refcam.shape = (N, 1, 2)
#reproj_err = np.mean(np.linalg.norm(reprojected_p_refcam - p_refcam, axis = -1))
reproj_err = np.mean(np.linalg.norm(reprojected_p_refcam - p_normrefcam, axis = -1))
print("averaged reprojection error [units: pixels]: {}".format(reproj_err))

secondcam2refcam_rot = cv2.Rodrigues(rvec_finetuned)[0]
secondcam2refcam = np.eye(4)
secondcam2refcam[:3, :3] = secondcam2refcam_rot                                             # shape = (3, 3)
secondcam2refcam[:3, 3:4] = tvec_finetuned                                                  # shape = (3, 1)
refcam2secondcam = np.linalg.inv(secondcam2refcam)

In [ ]:
print(refcam2secondcam)
np.save("/data2/ndsong/normrefcam2secondcam.npy", refcam2secondcam)

In [ ]:
#averaged reprojection error [units: pixels]: 0.031017746431295713
#averaged reprojection error [units: pixels]: 0.030368957641923437